# Path Planning

In [21]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from heapq import heappush, heappop
from numpy import array
from numpy.linalg import norm
import math
from math import atan


In [22]:
def heuristic(p1, p2):
    # Implement the Manhattan distance heuristic
    return abs(p1[0] - p2[0]) + abs(p1[1] - p2[1])

In [23]:
def path_functions(shortest_path):
    """
    Find the coefficient of the path slopes ( y = alpha*x + beta)

    input: 
        shortest_path: M X 2 (M = # of nodes in the shortest path), A list 
            of corner indices representing the shortest path from start to goal.

    output: 
        nodes_slopes = M x 4 (M = number of nodes in the shortest path), where 
        each row represents the node's coordinates along with the 
        alpha and beta coefficients.
    """

    M = shortest_path.shape[0]
    nodes_slopes = np.zeros((M, 4))
    for i in range(M-1):
        nodes_slopes[i,0] = shortest_path[i,0]
        nodes_slopes[i,1] = shortest_path[i,1]
        #check if the slope is not vertical
        if i != (M-1):
            if shortest_path[i+1,0] == shortest_path[i,0]:
                nodes_slopes[i,2] = float('inf')
                nodes_slopes[i,3] = float('inf')
            else :
                #find alpha
                nodes_slopes[i,2] = (shortest_path[i+1,1]-shortest_path[i,1])/(shortest_path[i+1,0]-shortest_path[i,0])
                #find beta
                nodes_slopes[i,3] = shortest_path[i+1,1]-shortest_path[i+1,0]*nodes_slopes[i,2]
            
    return nodes_slopes


In [24]:
def a_star_search(points,ex_path):
    """
    A* 

    input: 
        points: N x 2 array (N = (# of corners) + 2), where:
            - The first row represents the start position.
            - The second row represents the goal position.
            - The remaining rows contain the coordinates of the extended corners.
        ex_path: N X N, is equal to 1 if two corners are directly connected 
              by a line that does not cross any obstacle, and 0 otherwise.

    output: 
        shortest_path: M X 2 (M = # of corners in the shortest path), A list 
            of corner indices representing the shortest path from start to goal.
    """

    
    # Initialize the open set as a priority queue and add the start node
    open_set = []
    start_index = 0
    start = points[0,:]
    goal_index = 1
    goal = points[1,:]

    N = points.shape[0]
    distance_matrix = np.zeros((N, N))
    for i in range(N):
         for j in range(N):
              #calculate the distance between two corners
              distance_matrix[i,j] = norm(points[i,:]-points[j,])
    

    heappush(open_set, (heuristic(start, goal), 0, start_index))  # (f_cost, g_cost, position)

    # Initialize the came_from dictionary
    came_from = {}
    # Initialize g_costs dictionary with default value of infinity and set g_costs[start] = 0
    g_costs = {start_index: 0}
    # Initialize the explored set
    explored = set()
    operation_count = 0

    while open_set:
        # Pop the node with the lowest f_cost from the open set
        current_f_cost, current_g_cost, current_index = heappop(open_set)
        # Add the current node to the explored set
        explored.add(current_index)

        # For directly reconstruct path
        if current_index == goal_index:
            break

        # Get the neighbors of the current node 
        index_vector = ex_path[current_index,:]
        neighbors_index = np.nonzero(index_vector)[0]

        for index in neighbors_index:
             if  index not in explored:
                    # Calculate tentative_g_cost
                    tentative_g_cost = current_g_cost + distance_matrix[current_index,index]

                    # If this path to neighbor is better than any previous one
                    if index not in g_costs or tentative_g_cost < g_costs[index]:
                        # Update came_from, g_costs, and f_cost
                        came_from[index] = current_index
                        g_costs[index] = tentative_g_cost
                        f_cost = tentative_g_cost + heuristic(points[index,:], goal)
                        
                        # Add neighbor to open set
                        heappush(open_set, (f_cost, tentative_g_cost, index))
                        operation_count += 1

    # Reconstruct path
    if current_index == goal_index:
        path = []
        while current_index in came_from:
            path.append(points[current_index,:])
            current_index = came_from[current_index]
        path.append(start)
        np_path = np.array(path)
        return np_path[::-1]
    else:
        # If we reach here, no path was found
        return None
    

In [50]:
def path_direction(coordinates, nodes_slopes, segment_index):
    """
    Find on which slope the robot is so we can activate the motors 

    input: 
        coordinates: 2 x 3 array, where:
            - The first row represents the mean of x, y and theta coordinates.
            - The second row represents the standard deviation.
        nodes_slopes = M x 4 (M = number of nodes in the shortest path), where 
            each row represents the node's coordinates along with the 
            alpha and beta coefficients.

    output: 
        speed: 1 X 2 , contains the speed value for the left and right motor respectively. 
        segment_index: scalar indicationg the index of the segment the robot is on.
        end: scalar, if equal to 1 we're at the final destination
    """

    M = nodes_slopes.shape[0]
    speed = np.zeros((1, 2))
    y_mean = coordinates[0,1]
    x_mean = coordinates[0,1]
    theta_mean = coordinates[0,2]
    speed0 = 100
    theta_radians = math.radians(theta_mean) #si angle en degres
    tolerance_radians = 0.35  #2°
    end = 0


    print(segment_index)
    if segment_index == (M-2):
        end = 1

    #check if we're close to the end of the segment
    distance_segm = ((nodes_slopes[segment_index+1,1]-y_mean)**2 + (nodes_slopes[segment_index+1,0]-x_mean)**2)**0.5
    tolerance_norm = 0.01
    if distance_segm < tolerance_norm:
        print("lol")
        segment_index += 1
        if end == 1:
            speed[:] = [0,0]
            print(speed)
            return speed,segment_index,end



    #find slope between our coordinate and the end of the segment
    alpha = (nodes_slopes[segment_index+1,1]-y_mean)/(nodes_slopes[segment_index+1,0]-x_mean)
    #check if there's a big difference
    if abs(atan(alpha) - atan(nodes_slopes[segment_index,2])) < tolerance_radians:
        nodes_slopes[segment_index,2] = alpha

    if abs(theta_radians - atan(nodes_slopes[segment_index,2])) < tolerance_radians:
        speed[:] = [speed0,speed0]
        print(speed)
        return speed,segment_index,end

    if theta_radians > atan(nodes_slopes[segment_index,2]) :
        #clockwise rotation
        gradient = abs(theta_radians - atan(nodes_slopes[segment_index,2]))
        speed[0] = 20+gradient*100
        speed[1] = -speed[0]
        print(speed)
        return speed,segment_index,end
    if theta_radians < atan(nodes_slopes[segment_index,2]) :
        #counterclockwise rotation
        gradient = abs(theta_radians - atan(nodes_slopes[segment_index,2]))
        speed[1] = 20+gradient*100
        speed[0] = -speed[1]
        print(speed)
        return speed,segment_index,end

## test

In [51]:
points = np.array([[0, 0], [16, 0], [4, 0] , [8, 8], [4, 4]])
ex_path =  np.array([[1,0 ,1, 1, 1], [0,1 ,0, 1, 1], [1,0 ,1, 1, 1], [1,1 ,1, 1, 1], [1,1 ,1, 1, 1]])
path= a_star_search(points,ex_path)
nodes = path_functions(path)
coordinates = np.array([[2, 0, 30], [0.2, 0.2,0]])
segment_index = 0
speed,segment_index2,end = path_direction(coordinates, nodes, segment_index)



0
[[100. 100.]]
